<h1><center> Passage Based Question Answer Model </center></h1>

<h3><center> NLP PROJECT REVIEW 2 </center></h3>

In [ ]:
!pip install PyDrive

In [ ]:

# !pip install --user pandas==1.0.3

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

**Import Libraries**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import json
from pandas.io.json import json_normalize
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Load the dataset**

In [5]:
train = pd.read_json('/content/drive/MyDrive/Colab Notebooks/Squad/train-v2.0.json')
dev = pd.read_json('/content/drive/MyDrive/Colab Notebooks/Squad/dev-v2.0.json')


In [6]:
train.head()

,version,data
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'qas': [{'qu..."
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...


In [7]:
print("Shape of data =",train.shape)

Shape of data = (442, 2)


In [8]:
dev.head()

,version,data
0,v2.0,"{'title': 'Normans', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Computational_complexity_theory', '..."
2,v2.0,"{'title': 'Southern_California', 'paragraphs':..."
3,v2.0,"{'title': 'Sky_(United_Kingdom)', 'paragraphs'..."
4,v2.0,"{'title': 'Victoria_(Australia)', 'paragraphs'..."


In [9]:
print("Shape of dev data =", dev.shape)

Shape of dev data = (35, 2)


Put dev file in a dataframe to be readable

In [10]:
#This has multiple answers for same questions unlike traing data
def dtodf(dfile, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    #dfile: path to the squad json file.
    #record_path: path to deepest level in json file
    #verbose controls level of detail of progress messages
  
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(dfile).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    m['context'] = idx
    main = m[['id','question','context','answers']].set_index('id').reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [11]:
dfile = '/content/drive/MyDrive/Colab Notebooks/Squad/dev-v2.0.json'
record_path = ['data','paragraphs','qas','answers']
dev1 = dtodf(dfile=dfile,record_path=record_path)

Reading the json file
processing...


<ipython-input-10-c44eec628d0e>:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  js = pd.io.json.json_normalize(file , record_path )
<ipython-input-10-c44eec628d0e>:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  m = pd.io.json.json_normalize(file, record_path[:-1] )
<ipython-input-10-c44eec628d0e>:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  r = pd.io.json.json_normalize(file,record_path[:-2])


shape of the dataframe is (11873, 5)
Done


In [12]:
dev1.head()

,id,question,context,answers,c_id
0,56ddde6b9a695914005b9628,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,"[{'text': 'France', 'answer_start': 159}, {'te...",0
1,56ddde6b9a695914005b9629,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,"[{'text': '10th and 11th centuries', 'answer_s...",0
2,56ddde6b9a695914005b962a,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,"[{'text': 'Denmark, Iceland and Norway', 'answ...",0
3,56ddde6b9a695914005b962b,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,"[{'text': 'Rollo', 'answer_start': 308}, {'tex...",0
4,56ddde6b9a695914005b962c,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...,"[{'text': '10th century', 'answer_start': 671}...",0


**Put the training json file into a dataframe**

In [13]:
def ttodf(dfile, record_path = ['data','paragraphs','qas','answers'], verbose = 1):
    #dfile: path to the squad json file.
    #record_path: path to last level in the json file
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(dfile).read())
    if verbose:
        print("processing...")
    # parsing different levels in the json file
    js = pd.json_normalize(file , record_path )
    m = pd.json_normalize(file, record_path[:-1] )
    r = pd.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("\nJson has been successfully converted into a dataframe")
    return main

In [14]:
dfile = '/content/drive/MyDrive/Colab Notebooks/Squad/train-v2.0.json'
record_path = ['data','paragraphs','qas','answers']
train1 = ttodf(dfile=dfile,record_path=record_path)

Reading the json file
processing...


<ipython-input-13-809cd7b0e445>:19: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()



Json has been successfully converted into a dataframe


**Checking Structure of the dataset**

In [15]:
train1.head()

,index,question,context,text,answer_start,c_id
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0



**Text Data Pre-processing**

Data dimensions

In [16]:
print("Shape of data =", train1.shape)

Shape of data = (130319, 6)


Checking for Null values

In [17]:
train1.isnull().sum()
#no null values

index               0
question            0
context             0
text            43498
answer_start    43498
c_id                0
dtype: int64

In [18]:
#take a look at the text present in the columns
train1['question'].unique()

array(['When did Beyonce start becoming popular?',
       'What areas did Beyonce compete in when she was growing up?',
       "When did Beyonce leave Destiny's Child and become a solo singer?",
       ..., 'What is another name for anti-matter?',
       'Matter usually does not need to be used in conjunction with what?',
       'What field of study has a variety of unusual contexts?'],
      dtype=object)

In [19]:
train1['context'].unique()

array(['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
       'Following the disbandment of Destiny\'s Child in June 2005, she released her second solo album, B\'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and

In [20]:
train1['text'].unique()

array(['in the late 1990s', 'singing and dancing', '2003', ..., 'Oregon',
       'Minsk', 'Kathmandu Metropolitan City'], dtype=object)

**Cleaning Text Data for word embeddings**

When cleaning the text, we will perform the following task

1.   Convert text to lowercase
2.   Remove punctuations
3.   Remove extra space
4.   Stop words removal
5.   Stemming
6.   Lemmatization



1. Convert text to lowercase (because in NLP casing matters)


In [21]:
train1['clean_question']=train1['question'].apply(lambda x: x.lower() if isinstance(x, str) else str(x))
train1['clean_context']=train1['context'].apply(lambda x: x.lower() if isinstance(x, str) else str(x))
train1['clean_text']=train1['text'].apply(lambda x: x.lower() if isinstance(x, str) else str(x))

2. Removing punctuations

In [22]:
import re
import string
train1['clean_question']=train1['clean_question'].str.replace('[^\w\s]','')
train1['clean_context']=train1['clean_context'].str.replace('[^\w\s]','')
train1['clean_text']=train1['clean_text'].str.replace('[^\w\s]','')

<ipython-input-22-3e5e0fdaaa50>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train1['clean_question']=train1['clean_question'].str.replace('[^\w\s]','')
<ipython-input-22-3e5e0fdaaa50>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  train1['clean_context']=train1['clean_context'].str.replace('[^\w\s]','')
<ipython-input-22-3e5e0fdaaa50>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  train1['clean_text']=train1['clean_text'].str.replace('[^\w\s]','')


3. Removing extra space

In [23]:
train1['clean_question']=train1['clean_question'].apply(lambda x: re.sub(' +',' ',x) )
train1['clean_context']=train1['clean_context'].apply(lambda x: re.sub(' +',' ',x))
train1['clean_text']=train1['clean_text'].apply(lambda x: re.sub(' +',' ',x))

Checking

In [24]:
train1['clean_question'].unique()

array(['when did beyonce start becoming popular',
       'what areas did beyonce compete in when she was growing up',
       'when did beyonce leave destinys child and become a solo singer',
       ..., 'what is another name for antimatter',
       'matter usually does not need to be used in conjunction with what',
       'what field of study has a variety of unusual contexts'],
      dtype=object)

In [25]:
train1['clean_context'].unique()

array(['beyoncé giselle knowlescarter biːˈjɒnseɪ beeyonsay born september 4 1981 is an american singer songwriter record producer and actress born and raised in houston texas she performed in various singing and dancing competitions as a child and rose to fame in the late 1990s as lead singer of rb girlgroup destinys child managed by her father mathew knowles the group became one of the worlds bestselling girl groups of all time their hiatus saw the release of beyoncés debut album dangerously in love 2003 which established her as a solo artist worldwide earned five grammy awards and featured the billboard hot 100 numberone singles crazy in love and baby boy',
       'following the disbandment of destinys child in june 2005 she released her second solo album bday 2006 which contained hits déjà vu irreplaceable and beautiful liar beyoncé also ventured into acting with a golden globenominated performance in dreamgirls 2006 and starring roles in the pink panther 2006 and obsessed 2009 her 

In [26]:
train1['clean_text'].unique()

array(['in the late 1990s', 'singing and dancing', '2003', ..., 'oregon',
       'minsk', 'kathmandu metropolitan city'], dtype=object)

4. Removing stop words

In [27]:
from nltk.corpus import stopwords 
nltk.download('stopwords')
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 

def remove_stopwords(data):
  output_array=[]
  for sentence in train1['clean_text']:
    temp_list=[]
    for word in sentence.split():
        if word.lower() not in stopwords:
            temp_list.append(word)
        output_array.append(' '.join(temp_list))
  return output_array

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


5. Stemming

In [28]:
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language='english')

def stemmer(df):
  output_array=[]
  for sentence in df:
    temp_list=[]
    for word in sentence.split():
          temp_list.append(snowball.stem(word))
    output_array.append(' '.join(temp_list))
  output_array1 = pd.Series(output_array)
  return output_array1
data = train1['clean_text']
print(data.unique())
data = stemmer(data)
print(data.unique())

['in the late 1990s' 'singing and dancing' '2003' ... 'oregon' 'minsk'
 'kathmandu metropolitan city']
['in the late 1990s' 'sing and danc' '2003' ... 'oregon' 'minsk'
 'kathmandu metropolitan citi']


6. Lemmatization

In [29]:
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer() 

def lemmatize_words(df):
  output_array=[]
  for sentence in df:
    temp_list=[]
    for word in sentence.split():
          temp_list.append(lemmatizer.lemmatize(word))
    output_array.append(' '.join(temp_list))
  output_array1 = pd.Series(output_array)
  return output_array1

data = train1['clean_text']

print(data.unique())
data = lemmatize_words(data)
print(data.unique())
print("\nExample of lemmatization")
sentence = "The striped bats are hanging on their feet for best"

# Tokenize: Split the sentence into words
word_list = nltk.word_tokenize(sentence)
print(word_list)
#> ['The', 'striped', 'bats', 'are', 'hanging', 'on', 'their', 'feet', 'for', 'best']

# Lemmatize list of words and join
lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
print(lemmatized_output)
#> The striped bat are hanging on their foot for best


['in the late 1990s' 'singing and dancing' '2003' ... 'oregon' 'minsk'
 'kathmandu metropolitan city']
['in the late 1990s' 'singing and dancing' '2003' ... 'oregon' 'minsk'
 'kathmandu metropolitan city']

Example of lemmatization
['The', 'striped', 'bats', 'are', 'hanging', 'on', 'their', 'feet', 'for', 'best']
The striped bat are hanging on their foot for best


Statistically checking the data

In [30]:
#dropping columns
train1 = train1.drop(train1.columns[[0, 1, 2, 4, 5]], axis=1) 

In [31]:
train1.head()

,text,clean_question,clean_context,clean_text
0,in the late 1990s,when did beyonce start becoming popular,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,in the late 1990s
1,singing and dancing,what areas did beyonce compete in when she was...,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,singing and dancing
2,2003,when did beyonce leave destinys child and beco...,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,2003
3,"Houston, Texas",in what city and state did beyonce grow up,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,houston texas
4,late 1990s,in which decade did beyonce become famous,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,late 1990s


Transfer Learning


In [32]:
!pip install wandb  #monitor training in realtime
!pip install transformers  #make computer understand how humans understand language
!pip install seqeval  #used for sequence labelling evaluations,, evaluates NLP 
!pip install tensorboardx #used for visualizing matrix through histograms or plots
!pip install simpletransformers #used to quickly evaluate transformer model, uses SIMPLE one line code to train model

from simpletransformers.question_answering import QuestionAnsweringModel

train_args = {
    'learning_rate': 3e-5,
    'num_train_epochs': 2,  #no of training iterations
    'max_seq_length': 384, #max length of answer
    'doc_stride': 128,  #used to split input text into smaller seq, length of each seq=128
    'overwrite_output_dir': True, 
    'reprocess_input_data': False,
    'train_batch_size': 2, 
    'gradient_accumulation_steps': 8, #before model is implemented
}

model = QuestionAnsweringModel('bert', 'bert-base-cased', args=train_args, use_cuda=False)
#bert is model type and bert-base-cased is the model name
#specifiying pretrained model preserves casing of input text


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [33]:
#converting the json files to list of dictionary which is the suitable format to perform QA in simple transformers
train = [item for topic in train['data'] for item in topic['paragraphs'] ]

type(train)

list

In [34]:
len(train)

19035

In [35]:
#getting a random sample for training
import random
random.seed(3)
randomtrain=random.sample(train, 5000)


In [36]:
len(randomtrain)

5000

In [37]:
remain = [ele for ele in train if ele not in randomtrain]
len(remain) 

14035

In [38]:
#getting a random sample for testing
random.seed(6)
randomtest=random.sample(remain, 1000)

In [ ]:
import sklearn
model.train_model(randomtrain, acc=sklearn.metrics.accuracy_score)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/17470 [00:00<?, ?it/s]

In [ ]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))
#into binary format (serializing data)

! cp model.pkl "/content/drive/MyDrive/Colab Notebooks/Squad"
model = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Squad/model.pkl', 'rb'))

Evaluation

In [ ]:
import sklearn
result, text= model.eval_model(randomtest, acc=sklearn.metrics.accuracy_score, verbose= True)
print(result)#contains evaluation result
print(text)#a dict containing the the correct_text, similar_text and incorrect text

Prediction

In [ ]:
import sklearn
dev = [item for topic in dev['data'] for item in topic['paragraphs'] ]
predtn1, raw_outputs = model.predict(dev, n_best_size=2)#return a list of dict+ containg each question mapped to its answer and a list of dicts of question id mapped to probability score of the answer

predtn1[0]



In [ ]:
raw_outputs[0]

In [ ]:
context_text = "Linear regression is used for predicting quantitative values, such as an individual’s salary. In order to predict qualitative values, such as whether a patient survives or dies, or whether the stock market increases or decreases, Fisher proposed linear discriminant analysis in 1936."
predtn2, raw_outputs = model.predict(
    [
        {
            "context": context_text,
            "qas": [
                {
                    "question": "Who proposed linear discriminant analysis?",
                    "id": "0",
                }
            ],
        }
    ],
n_best_size=2)

In [ ]:
print(predtn2)#return a list of dict containg each question mapped to its answer 

print(raw_outputs)#a list of dicts of question id mapped to probability score of the answer


In [ ]:
context_text = "Thomas Alva Edison was an American inventor and businessman who has been described as America's greatest inventor.One of his inventions, is the phonograph"
predtn3, raw_outputs = model.predict(
    [
        {
            "context": context_text,
            "qas": [
                {
                    "question": "Who invented phonograph?",
                    "id": "0",
                }
            ],
        }
    ],
n_best_size=2)

In [ ]:
print(predtn3)#return a list of dict containg each question mapped to its answer 
print(raw_outputs)#a list of dicts of question id mapped to probability score of the answer

In [ ]:
context_text = "Mary drove for 3 hours to work in the morning and 5 hours in the evening to her house"
predtn4, raw_outputs = model.predict(
    [
        {
            "context": context_text,
            "qas": [
                {
                    "question": "How many hours did Mary drive for today?",
                    "id": "0",
                }
            ],
        }
    ],
n_best_size=2)

In [ ]:
print(predtn4)#return a list of dict containg each question mapped to its answer 
print(raw_outputs)#a list of dicts of question id mapped to probability score of the answer

In [ ]:
import os
 
paragraph = "Caffeine, the stimulant in coffee, has been called \u201Cthe most widely used psychoactive substance on Earth.\u201D Snyder, Daly, and Bruns have recently proposed that caffeine affects behavior by countering the activity in the human brain of a naturally occurring chemical called adenosine. Adenosine normally depresses neuron firing in many areas of the brain. It apparently does this by inhibiting the release of neurotransmitters, chemicals that carry nerve impulses from one neuron to the next.  Like many other agents that affect neuron firing, adenosine must first bind to specific receptors on neuronal membranes. There are at least two classes of these receptors, which have been designated A1 and A2. Snyder et al. propose that caffeine, which is structurally similar to adenosine, is able to bind to both types of receptors, which prevents adenosine from attaching there and allows the neurons to fire more readily than they otherwise would.  For many years, caffeine\u2019s effects have been attributed to its inhibition of the production of phosphodiesterase, an enzyme that breaks down the chemical called cyclic AMP. A number of neurotransmitters exert their effects by first increasing cyclic AMP concentrations in target neurons. Therefore, prolonged periods at the elevated concentrations, as might be brought about by a phosphodiesterase inhibitor, could lead to a greater amount of neuron firing and, consequently, to behavioral stimulation. But Snyder et al. point out that the caffeine concentrations needed to inhibit the production of phosphodiesterase in the brain are much higher than those that produce stimulation. Moreover, other compounds that block phosphodiesterase\u2019s activity are not stimulants.  To buttress their case that caffeine acts instead by preventing adenosine binding, Snyder et al. compared the stimulatory effects of a series of caffeine derivatives with their ability to dislodge adenosine from its receptors in the brains of mice. \u201CIn general,\u201D they reported, \u201Cthe ability of the compounds to compete at the receptors correlates with their ability to stimulate locomotion in the mouse; i.e., the higher their capacity to bind at the receptors, the higher their ability to stimulate locomotion.\u201D Theophylline, a close structural relative of caffeine and the major stimulant in tea, was one of the most effective compounds in both regards.  There were some apparent exceptions to the general correlation observed between adenosine receptor binding and stimulation. One of these was a compound called 3-isobutyl-1-methylxanthine (IBMX), which bound very well but actually depressed mouse locomotion. Snyder et al. suggest that this is not a major stumbling block to their hypothesis. The problem is that the compound has mixed effects in the brain, a not unusual occurrence with psychoactive drugs. Even caffeine, which is generally known only for its stimulatory effects, displays this property, depressing mouse locomotion at very low concentrations and stimulating it at higher ones." #@param {type:"string"}
question = "what is caffeine's effects attributed to?" #@param {type:"string"}
 
algorithm = 'BERT' #@param{type:"string"}

model = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Squad/model.pkl', 'rb'))

predtn5, raw_outputs = model.predict(
    [
        {
            "context": paragraph,
            "qas": [
                {
                    "question": question,
                    "id": "0",
                }
            ],
        }
    ],
n_best_size=2)

print(predtn5[0]['answer'])
print(raw_outputs[0]['probability'])

print("\nMost likely answer is: ",predtn5[0]['answer'][0])
